# 🎙️ PersonaPlex Demo — Proof of Concept

**Proyecto:** PersonaPlex Callbot (Fase 1)

Este notebook prueba PersonaPlex en Google Colab.

### ⚠️ Requisitos de GPU:
- **T4 (free tier): NO funciona** — 15GB VRAM + 12GB RAM insuficiente. El proceso es `Killed` por OOM.
- **L4 o superior: Recomendado** — Necesitas Colab Pro o créditos de GPU.
- **A100: Ideal** — Es el hardware de referencia de NVIDIA.

### Antes de empezar:
1. ✅ Acepta la licencia del modelo: [nvidia/personaplex-7b-v1](https://huggingface.co/nvidia/personaplex-7b-v1)
2. ✅ Ten tu HuggingFace token listo
3. ✅ Selecciona Runtime → **L4 GPU o superior**

---

## 1️⃣ Verificar GPU

In [ ]:
!nvidia-smi
import torch
print(f"\n✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {vram_gb:.1f} GB")
    if vram_gb < 20:
        print(f"\n⚠️  {gpu_name} tiene solo {vram_gb:.0f}GB VRAM.")
        print("   PersonaPlex necesita ~17GB. Se usará --cpu-offload.")
        print("   Si la GPU tiene <16GB VRAM y el sistema <16GB RAM, puede fallar (OOM/Killed).")
        print("   T4 (15GB) en Colab Free NO funciona. Usa L4 o superior.")

## 2️⃣ Instalar dependencias del sistema

In [ ]:
!apt-get update -qq && apt-get install -y -qq libopus-dev > /dev/null 2>&1
print("✅ libopus-dev instalado")

## 3️⃣ Clonar el repositorio e instalar PersonaPlex

In [ ]:
!git clone https://github.com/NVIDIA/personaplex.git
%cd personaplex
!pip install -q moshi/.
!pip install -q accelerate  # Needed for --cpu-offload
print("\n✅ PersonaPlex instalado")

## 4️⃣ Autenticación con HuggingFace

Ingresa tu token de HuggingFace (necesario para descargar los weights).

In [ ]:
import os
from getpass import getpass

# Opción 1: Ingreso manual
hf_token = getpass("🔑 Ingresa tu HuggingFace Token: ")
os.environ["HF_TOKEN"] = hf_token

# Verificar
print(f"✅ Token configurado ({len(hf_token)} chars)")

## 5️⃣ Test Offline — Modo Asistente

Primer test: usar el audio de prueba incluido con la voz NATF2 (femenina natural).

⚠️ Usamos `--cpu-offload` si la GPU tiene <20GB VRAM.

In [ ]:
%%time
import torch
vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
offload = "--cpu-offload" if vram_gb < 20 else ""
print(f"GPU VRAM: {vram_gb:.0f}GB → {'cpu-offload ON' if offload else 'full GPU'}")

!python -m moshi.offline \
    --voice-prompt "NATF2.pt" \
    --input-wav "assets/test/input_assistant.wav" \
    --seed 42424242 \
    --output-wav "output_assistant.wav" \
    --output-text "output_assistant.json" \
    {offload} 2>&1

import os
if os.path.exists("output_assistant.wav"):
    print(f"\n✅ Audio generado: output_assistant.wav ({os.path.getsize('output_assistant.wav')} bytes)")
else:
    print("\n❌ FALLÓ: output_assistant.wav no fue generado.")
    print("   Probable causa: OOM. Esta GPU no tiene suficiente memoria.")
    print("   Solución: Usar una GPU con ≥20GB VRAM (L4, A100) o probar localmente con personaplex-mlx.")

### Reproducir resultado

In [ ]:
import os
from IPython.display import Audio, display
import json

print("🎧 Audio de entrada (lo que 'escuchó' PersonaPlex):")
display(Audio("assets/test/input_assistant.wav"))

if os.path.exists("output_assistant.wav"):
    print("\n🎙️ Respuesta de PersonaPlex (24kHz):")
    display(Audio("output_assistant.wav", rate=24000))
else:
    print("\n❌ No hay audio de salida — la generación falló. Ver celda anterior.")

if os.path.exists("output_assistant.json"):
    print("\n📝 Transcripción de la respuesta:")
    with open("output_assistant.json") as f:
        transcript = json.load(f)
        print(json.dumps(transcript, indent=2))
else:
    print("\n📝 No hay transcripción disponible.")

## 6️⃣ Test Offline — Modo Customer Service

Segundo test: modo servicio al cliente con voz masculina NATM1 y un prompt de rol.

In [ ]:
%%time
import torch, os
vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
offload = "--cpu-offload" if vram_gb < 20 else ""

!python -m moshi.offline \
    --voice-prompt "NATM1.pt" \
    --text-prompt "$(cat assets/test/prompt_service.txt)" \
    --input-wav "assets/test/input_service.wav" \
    --seed 42424242 \
    --output-wav "output_service.wav" \
    --output-text "output_service.json" \
    {offload} 2>&1

if os.path.exists("output_service.wav"):
    print(f"\n✅ Audio generado: output_service.wav ({os.path.getsize('output_service.wav')} bytes)")
else:
    print("\n❌ FALLÓ: output_service.wav no fue generado (OOM probable).")

In [ ]:
import os
from IPython.display import Audio, display
import json

print("🎧 Audio de entrada (cliente):")
display(Audio("assets/test/input_service.wav"))

if os.path.exists("output_service.wav"):
    print("\n🎙️ Respuesta de PersonaPlex (agente):")
    display(Audio("output_service.wav", rate=24000))
else:
    print("\n❌ No hay audio de salida.")

if os.path.exists("output_service.json"):
    print("\n📝 Transcripción:")
    with open("output_service.json") as f:
        transcript = json.load(f)
        print(json.dumps(transcript, indent=2))
else:
    print("\n📝 No hay transcripción disponible.")

## 7️⃣ Probar diferentes voces

Iteramos sobre varias voces para comparar calidad y estilo.

In [ ]:
import subprocess, json, os, torch
from IPython.display import Audio, display

vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
offload_args = ["--cpu-offload"] if vram_gb < 20 else []

voices_to_test = ["NATF0", "NATF2", "NATM0", "NATM2", "VARF1", "VARM1"]
results = {}

for voice in voices_to_test:
    print(f"\n{'='*50}")
    print(f"🎤 Probando voz: {voice}")
    print(f"{'='*50}")
    
    out_wav = f"output_voice_{voice}.wav"
    out_json = f"output_voice_{voice}.json"
    
    result = subprocess.run([
        "python", "-m", "moshi.offline",
        "--voice-prompt", f"{voice}.pt",
        "--input-wav", "assets/test/input_assistant.wav",
        "--seed", "42424242",
        "--output-wav", out_wav,
        "--output-text", out_json,
    ] + offload_args, capture_output=True, text=True)
    
    if result.returncode == 0 and os.path.exists(out_wav):
        print(f"✅ {voice} completado")
        display(Audio(out_wav, rate=24000))
        
        if os.path.exists(out_json):
            with open(out_json) as f:
                transcript = json.load(f)
                results[voice] = transcript
                print(f"📝 {json.dumps(transcript, indent=2)[:300]}")
    else:
        print(f"❌ Error con {voice}: {result.stderr[:300] if result.stderr else 'Killed (OOM)'}")

print(f"\n\n🏁 Voces completadas: {len(results)}/{len(voices_to_test)}")

## 8️⃣ Probar prompt personalizado (Callbot)

Simulamos un escenario de callbot tipo OnBotGo.

In [ ]:
# Prompt personalizado estilo callbot
custom_prompt = """You work for TechSupport Pro which is a technical support service and your name is Sarah. \
Information: You help customers troubleshoot internet connectivity issues. \
Common solutions: restart router (wait 30 seconds), check cable connections, \
run speed test at speedtest.net. If issue persists, schedule technician visit \
(next available: tomorrow 2-4 PM or Thursday 9-11 AM). \
Service costs: Basic plan $29.99/month, Premium $49.99/month with priority support."""

# Guardar prompt
with open("custom_prompt.txt", "w") as f:
    f.write(custom_prompt)

print("Prompt guardado ✅")
print(f"\n📋 Prompt:\n{custom_prompt}")

In [ ]:
%%time
import torch, os
vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
offload = "--cpu-offload" if vram_gb < 20 else ""

!python -m moshi.offline \
    --voice-prompt "NATF2.pt" \
    --text-prompt "$(cat custom_prompt.txt)" \
    --input-wav "assets/test/input_service.wav" \
    --seed 42424242 \
    --output-wav "output_callbot.wav" \
    --output-text "output_callbot.json" \
    {offload} 2>&1

if os.path.exists("output_callbot.wav"):
    print(f"\n✅ Callbot test completado ({os.path.getsize('output_callbot.wav')} bytes)")
else:
    print("\n❌ FALLÓ (OOM probable).")

In [ ]:
import os
from IPython.display import Audio, display
import json

if os.path.exists("output_callbot.wav"):
    print("🎙️ Respuesta del callbot:")
    display(Audio("output_callbot.wav", rate=24000))
else:
    print("❌ No hay audio de salida.")

if os.path.exists("output_callbot.json"):
    print("\n📝 Transcripción:")
    with open("output_callbot.json") as f:
        transcript = json.load(f)
        print(json.dumps(transcript, indent=2))
else:
    print("📝 No hay transcripción disponible.")

## 9️⃣ Métricas de Rendimiento

Capturamos tiempos y uso de recursos para evaluar viabilidad.

In [ ]:
import time, os, subprocess, torch

vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
offload_args = ["--cpu-offload"] if vram_gb < 20 else []

print("📊 Benchmark: Latencia de generación offline")
print("="*50)

start = time.time()
result = subprocess.run([
    "python", "-m", "moshi.offline",
    "--voice-prompt", "NATF2.pt",
    "--input-wav", "assets/test/input_assistant.wav",
    "--seed", "12345",
    "--output-wav", "benchmark_output.wav",
    "--output-text", "benchmark_output.json",
] + offload_args, capture_output=True, text=True)
elapsed = time.time() - start

if os.path.exists("benchmark_output.wav"):
    import wave
    with wave.open("assets/test/input_assistant.wav") as w:
        input_duration = w.getnframes() / w.getframerate()

    with wave.open("benchmark_output.wav") as w:
        output_duration = w.getnframes() / w.getframerate()

    gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
    print(f"⏱️  Tiempo total: {elapsed:.1f}s")
    print(f"🎵 Duración input: {input_duration:.1f}s")
    print(f"🎵 Duración output: {output_duration:.1f}s")
    print(f"⚡ RTF (Real Time Factor): {elapsed/output_duration:.2f}x")
    print(f"   (< 1.0 = más rápido que tiempo real)")
    !nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader
    print(f"\n{'='*50}")
    print("📋 Resumen para PROJECT.md:")
    print(f"   GPU: {gpu_name} + {'cpu-offload' if offload_args else 'full GPU'}")
    print(f"   RTF: {elapsed/output_duration:.2f}x")
    print(f"   Tiempo gen: {elapsed:.1f}s para {output_duration:.1f}s de audio")
else:
    print(f"❌ Benchmark falló — no se generó audio.")
    print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
    print(f"   VRAM: {vram_gb:.0f}GB")
    print(f"   Resultado: OOM/Killed después de {elapsed:.0f}s")
    print(f"   Conclusión: Esta GPU no es suficiente para PersonaPlex.")

---

## ✅ Resultados Fase 1

| Métrica | Valor |
|---------|-------|
| GPU usada | *(llenar)* |
| cpu-offload | Sí/No |
| RTF | *(llenar)* |
| Calidad audio | *(evaluar subjetivamente)* |
| Mejor voz femenina | *(llenar)* |
| Mejor voz masculina | *(llenar)* |
| Adherencia al prompt | *(evaluar)* |
| Solo inglés | ✅ Confirmado |

### ⚠️ Hallazgo: T4 Colab Free NO funciona
- T4 (15GB VRAM, ~12GB RAM) → Proceso `Killed` por OOM al cargar moshi (16.7GB)
- Mínimo requerido: GPU con ≥20GB VRAM o sistema con ≥24GB RAM para cpu-offload
- Alternativa local: personaplex-mlx en Apple Silicon (RAM unificada)

### Próximos pasos:
- [ ] Probar en Mac mini con personaplex-mlx (Fase 1, tarea 2)
- [ ] Evaluar si RTF < 1.0 es alcanzable en local
- [ ] Diseñar integración con Twilio (Fase 2)